#### Import

In [1]:
import pandas as pd
import calendar
import os
import re

In [ ]:
year=2018
path_to_file = 'Finance '+str(year)+'.xlsx' #Old format file

In [ ]:
workbook = pd.ExcelFile(path_to_file) #print(workbook.sheet_names)
sheet = workbook.sheet_names[0]
month_num=list(calendar.month_name).index(sheet) #print(sheet, month_num)

In [4]:
df = pd.read_excel(workbook, sheet)

#### 1. Remove total, category and useless rows

In [ ]:
cat_row_ind = df.index[df.iloc[:,1] == 'Cost'][0]
total_row_ind = df.index[df.iloc[:,0] == 'Total'][0]
df1 = df.drop(df.index[total_row_ind:])
df2 = df1.drop(cat_row_ind)

#### 2. Fix column names and switch to dates

In [6]:
cols = df2.columns.tolist()
coly = cols.copy()

In [7]:
for r in range(len(cols)//2): #cols[2][:-(len(sheet)+1)]
    coly[2*r] = cols[2*r][-2:].strip()+'/'+str(month_num)+'/'+str(year)
    coly[2*r+1] = coly[2*r]
df2.columns = coly
#df2

#### 3. Reshape, drop Nan, and convert to costs to floats (removing £)

In [8]:
reshape = []
for date in df2.columns.unique():
    items = df2[date].iloc[:, 0].tolist()  # 'item' values
    costs = df2[date].iloc[:, 1].tolist()  # 'cost' values
    for item, cost in zip(items, costs):
        reshape.append({"Date": date, "Item": item, "Cost": cost})

df3 = pd.DataFrame(reshape).dropna(subset=['Item', 'Cost'])
#df3["Cost"] = df3["Cost"].str.replace(',', '').str.replace('£', '',regex=False).astype(float)
df3["Cost"] = df3["Cost"].round(2)# Limit to 2 decimal values
#df3

#### 4. Export back to .xlsx

In [9]:
#month_name = calendar.month_name[month_num]
month_name = sheet
filename = month_name+str(year)
df3.to_excel(filename+'.xlsx',index=False)

#### 5. Full function (for sheets)

In [ ]:
def finance_formatter_sheet(workbook,year,sheet):
    month_num=str(list(calendar.month_name).index(sheet))
    if len(month_num) == 1:
        month_num = '0'+month_num
    df = pd.read_excel(workbook, sheet)
    # remove total, category and useless rows
    cat_row_ind = df.index[df.iloc[:,1] == 'Cost'][0]
    total_row_ind = df.index[df.iloc[:,0] == 'Total'][0]
    df = df.drop(df.index[total_row_ind:])
    df = df.drop(cat_row_ind)
    # fix column names and switch to dates
    cols = df.columns.tolist()
    for r in range(len(cols)//2):
        day = cols[2*r][-2:].strip()
        if len(day) == 1:
            day = '0'+day
        cols[2*r] = day+'/'+month_num+'/'+str(year)
        cols[2*r+1] = cols[2*r]
    df.columns = cols
    # reshape, drop nan, and convert to costs to floats (dropping £)
    reshape = []
    for date in df.columns.unique():
        items = df[date].iloc[:, 0].tolist()
        costs = df[date].iloc[:, 1].tolist()
        for item, cost in zip(items, costs):
            reshape.append({"Date": date, "Item": item, "Cost": cost})
    
    df = pd.DataFrame(reshape).dropna(subset=['Item', 'Cost'])
    df = df.dropna(how='all')
    df["Cost"] = df["Cost"].round(2)

    return df

#### 6. Workbook expansion + export

In [8]:
def finance_formatter_workbook(path_to_folder,year):
    path_to_file = str(path_to_folder)+' '+str(year)+'.xlsx'
    workbook = pd.ExcelFile(path_to_file)
    sheets = [s for s in workbook.sheet_names if s in list(calendar.month_name)] # Make sure the sheets are named by month
    output_name = str(year)+'.xlsx' #'Finance '+
    dataframes = []
    for sheet in sheets:
        df = finance_formatter_sheet(workbook,year,sheet)
        dataframes.append(df)
    with pd.ExcelWriter(output_name) as writer: #'New Format/'+
        for df,sheet in zip(dataframes,sheets):
            df.to_excel(writer,sheet_name=sheet,index=False)

In [ ]:
dir_list = os.listdir() #add your path
names = pd.DataFrame(dir_list)
names = names[names[0].str.contains('Finance')]
names = sorted(names[0].tolist())
for l in range(len(names)):
    line = re.sub('.xlsx', '', names[l])
    line = re.sub('Finance ', '', line)
    names[l] = line
names.sort(reverse=False)
names = list(map(int, names))
years = [year for year in names if year in list(range(0,2100))] # guarantee files are named by year